# 종목별 연도별 상승일/하락일 계산

상승하락일 기준 수익률 1%, -1%

In [1]:
import FinanceDataReader as fdr
fdr.__version__

'0.6.0'

In [2]:
import numpy as np 
from statsmodels import regression 
import statsmodels.api as sm 
import matplotlib.pyplot as plt 
import math 
import pandas as pd 
import pandas_datareader as pdr

In [3]:
# import FinanceDataReader as fdr

# 한국거래소 상장종목 전체
df_krx = fdr.StockListing('KRX')
df_krx.head()

,Symbol,Name,Sector,Industry
0,001040,CJ,기타 금융업,지주회사
1,011150,CJ씨푸드,기타 식품 제조업,"수산물(어묵,맛살)가공품 도매,원양수산업,수출입"
2,012630,HDC,건물 건설업,"토목공사,건축공사,아파트분양사업,재개발/재건축사업"
3,082740,HSD엔진,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진"
4,001390,KG케미칼,기초 화학물질 제조업,"콘크리트혼화제, 비료, 친환경농자재, 수처리제"


### 1. 종목별 기간 조회 함수 정의

In [4]:
def stock_select(code,strt_year='2012', end_date='2018-09-30'):

    stock_df = fdr.DataReader(code,strt_year,end_date)

    # Date를 인덱스에서 제거하고 항목으로 사용
    stock_df = stock_df.reset_index()

    # Date에서 연도를 추출함
    stock_df['Year'] = stock_df.Date.map(lambda x : x.year )
    stock_df['Code'] = code

    return stock_df


In [10]:
stock_df.values

array([[Timestamp('2016-01-03 00:00:00'), 1260000, 1260000, ..., 0.0,
        2016, '005930'],
       [Timestamp('2016-01-04 00:00:00'), 1205000, 1260000, ..., -0.0437,
        2016, '005930'],
       [Timestamp('2016-01-05 00:00:00'), 1208000, 1202000, ..., 0.0025,
        2016, '005930'],
       ...,
       [Timestamp('2016-12-28 00:00:00'), 35760, 35840, ...,
        -0.0060999999999999995, 2016, '005930'],
       [Timestamp('2016-12-29 00:00:00'), 36040, 35420, ...,
        0.0078000000000000005, 2016, '005930'],
       [Timestamp('2017-01-01 00:00:00'), 1802000, 1802000, ..., 49.0,
        2017, '005930']], dtype=object)

In [11]:
stock_df['Year'].map(lambda x:min(x))

TypeError: 'int' object is not iterable

In [6]:
stock_df = stock_select('005930', '2016','2017')

In [ ]:
year_days_df = pd.DataFrame()
    
    
for x in range(0,10):  #len(df_krx)  체크
    # 종목 데이터 추출
    code = df_krx.Symbol.iloc[x]
    name = df_krx.Name.iloc[x]

    
    # 초기화
    temp_days_df = pd.DataFrame()
    year_days_dict = {}
    year_days_dict['Code'] = code
    year_days_dict['Name'] = name    

    # 연도별 일수 계산
    for year in range(2012,2018+1):  # 변경확인 


        # 종목별 기간별 데이터 추출
        try:
            if year == 2018: # 변경확인
                end_year = '2018-09-30' # 변경확인
            else:
                end_year = year+1
            stock_df = stock_select(code, str(year),str(end_year))
            target_df = stock_df[stock_df['Year'] == year]
            print(code, name, target_df['Date'].map(lambda x:min(x)), "-", target_df['Date'].map(lambda x:max(x)), "추출")
            
        except:
            print(code, name, year, "종목정보 추출 실패")
            continue
        
        try:
            # 상승/하락일 카운팅
            up_days = len(target_df.query("Change >= 0.01")) 
            down_days = len(target_df.query("Change < -0.01")) 
            tot_days = len(target_df)
            
         
            
        except:
            up_days = ""         
            down_days = ""
            tot_days = ""
        
        # 연도별 일수 변수에 저장
        year_days_dict["up_" + str(year)] = up_days
        year_days_dict["down_" + str(year)] = down_days
        year_days_dict["tot_" + str(year)] = tot_days   
            
    # 연도별 데이터를 임시데이터프레임에 저장(1개종목)
    temp_days_df = pd.Series(year_days_dict).to_frame()
    temp_days_df = temp_days_df.T



    try:
        year_days_df = pd.concat([year_days_df, temp_days_df]) # 이전 데이터와 연결
        print(x, "번 : ", code, name, "Days 생성완료")
    except:
        print(x, "번 : ", code,name, "Days 생성실패")
        break
    


In [7]:
year_days_df

,Code,Name,down_2012,down_2013,down_2014,down_2015,down_2016,down_2017,down_2018,tot_2012,...,tot_2016,tot_2017,tot_2018,up_2012,up_2013,up_2014,up_2015,up_2016,up_2017,up_2018
0,001040,CJ,48,79,70,80,84,72,54,248,...,267,261,198,61,74,76,91,62,61,38
0,011150,CJ씨푸드,66,55,43,72,70,39,41,248,...,260,254,196,76,43,41,78,63,36,38
0,012630,HDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,082740,HSD엔진,74,76,80,83,97,96,51,248,...,277,261,182,65,68,64,57,89,85,69
0,001390,KG케미칼,44,50,64,78,44,54,65,248,...,250,260,205,52,60,68,71,51,55,68
0,010060,OCI,76,59,81,90,80,61,66,248,...,283,284,218,55,66,56,81,80,70,52
0,002360,SH에너지화학,89,74,58,85,74,50,50,248,...,282,285,213,99,63,67,78,68,55,40
0,001740,SK네트웍스,78,69,71,85,51,59,46,248,...,275,272,205,68,64,80,70,60,48,37
0,011810,STX,77,113,97,95,112,18,25,248,...,275,39,67,62,80,72,55,91,18,12
0,071970,STX중공업,82,112,45,59,117,60,62,248,...,256,188,211,67,83,32,33,81,47,42


In [8]:
target_df

,Date,Close,Open,High,Low,Volume,Change,Year,Code
0,2018-01-01,4605,4605,4605,4605,0.0,0.0000,2018,071970
1,2018-01-02,4730,4620,4755,4600,279610.0,0.0271,2018,071970
2,2018-01-03,4775,4735,4840,4725,273840.0,0.0095,2018,071970
3,2018-01-04,4740,4750,4830,4700,254280.0,-0.0073,2018,071970
4,2018-01-05,4945,4740,5040,4720,879360.0,0.0432,2018,071970
5,2018-01-08,4845,4900,4900,4800,322850.0,-0.0202,2018,071970
6,2018-01-09,4850,4955,4995,4845,393920.0,0.0010,2018,071970
7,2018-01-10,4985,4885,4995,4815,413770.0,0.0278,2018,071970
8,2018-01-11,4950,5000,5040,4910,323560.0,-0.0070,2018,071970
9,2018-01-12,4860,5000,5000,4835,264620.0,-0.0182,2018,071970
